In [1]:
###standalone...why it works on local?
import numpy as np
from glob import glob
from sklearn.datasets import load_files 
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image                        
from keras.preprocessing.image import img_to_array  
from keras.applications import InceptionV3 
from keras.applications.inception_v3 import preprocess_input
from keras.models import Model
from keras.optimizers import SGD  
from PIL import ImageFile    
ImageFile.LOAD_TRUNCATED_IMAGES = True 

targetsize = (299,299)
# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('dogImages/train')
valid_files, valid_targets = load_dataset('dogImages/valid')
test_files, test_targets = load_dataset('dogImages/test')

def path_to_tensor(img_path,targetsize=targetsize):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=targetsize)
    # convert PIL.Image.Image type to 3D tensor with shape (w, h, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, w, h, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths,targetsize=targetsize):
    list_of_tensors = [path_to_tensor(img_path,targetsize) for img_path in img_paths]
    return np.vstack(list_of_tensors)


def bestdog_predict_breed(modelname, img_file, target_size):
    img_file = image.load_img(img_file, target_size=(299, 299))
    if img_file.size != target_size:
        img_file = img_file.resize(target_size)
    x = image.img_to_array(img_file)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = modelname.predict(x)
    return  np.argmax(preds[0])


# create and configure augmented image generator
datagen_train = ImageDataGenerator(
    width_shift_range=0.1,   # randomly shift images horizontally (10% of total width)
    height_shift_range=0.1,  # randomly shift images vertically (10% of total height)
    horizontal_flip=True)    # randomly flip images horizontally

# create and configure augmented image generator
datagen_valid = ImageDataGenerator(
    width_shift_range=0.1,   # randomly shift images horizontally (10% of total width)
    height_shift_range=0.1,  # randomly shift images vertically (10% of total height)
    horizontal_flip=True)    # randomly flip images horizontally

# fit augmented image generator on data
# pre-process the data for Keras to 4D tensor
train_tensors = paths_to_tensor(train_files,targetsize).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files,targetsize).astype('float32')/255
test_tensors = paths_to_tensor(test_files,targetsize).astype('float32')/255
 
datagen_train.fit(train_tensors)
datagen_valid.fit(valid_tensors)

##########

def add_new_last_layer(base_model, nb_classes,fullyconnected_size):
    # add hidden layer 
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(fullyconnected_size, activation='relu')(x) 
    predictions = Dense(nb_classes, activation='softmax')(x) 
    model = Model(input=base_model.input, output=predictions)
    return model

#Freeze all layers and compile the model
def setup_to_transfer_learn(model, base_model):   
    for layer in base_model.layers:
        layer.trainable = False
    model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

#fine tune the model....retrain model after IV3_LAYERS_TO_FREEZE layers
def setup_to_finetune(model,IV3_LAYERS_TO_FREEZE):
    for layer in model.layers[:IV3_LAYERS_TO_FREEZE]:
        layer.trainable = False
    for layer in model.layers[IV3_LAYERS_TO_FREEZE:]:
        layer.trainable = True
    model.compile(optimizer=SGD(lr=0.0001, momentum=0.9),loss='categorical_crossentropy')


    
    
    
batch_size = 32
epochs = 1 
nb_classes = 133
fullyconnected_size=1024
iv3_layers_to_freeze = 248

base_model = InceptionV3(weights='imagenet', include_top=False) #include_top=False excludes final Fully Connected layer
#base_model.output.shape......(?, ?, ?, 2048)
bestmodel = add_new_last_layer(base_model, nb_classes, fullyconnected_size)

# transfer learning
setup_to_transfer_learn(bestmodel, base_model)

##### train model
bestmodel.fit_generator(datagen_train.flow(train_tensors, train_targets, batch_size=batch_size),
                    steps_per_epoch=len(train_files) // batch_size,
                    epochs=3, verbose=1, 
                    #callbacks=[checkpointer],
                    validation_data=datagen_valid.flow(valid_tensors, valid_targets, batch_size=batch_size),
                    validation_steps=len(valid_files) // batch_size)
#save model with just argumented images, this will see any improvement from original base_model                       
bestmodel.save("bestofbest0_InceptionV3.model")
 
    
best_predictions = []
best_output=np.zeros((len(test_files),133))
for img in test_files:
    best_predictions.append(bestdog_predict_breed(bestmodel,img, (299,299)))
   
  

test_accuracy = 100*np.sum(np.array(best_predictions) ==
                           np.argmax(test_targets, axis=1))/len(best_predictions)
print('\nTest accuracy: %.4f%%' % test_accuracy)


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:85: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Epoch 1/3
208/208 [==============================] - 3089s 15s/step - loss: 2.1098 - acc: 0.5138 - val_loss: 0.7792 - val_acc: 0.7641
Epoch 2/3
208/208 [==============================] - 3077s 15s/step - loss: 0.8985 - acc: 0.7260 - val_loss: 0.5449 - val_acc: 0.8156
Epoch 3/3
208/208 [==============================] - 3044s 15s/step - loss: 0.7133 - acc: 0.7785 - val_loss: 0.6119 - val_acc: 0.8060

Test accuracy: 79.9043%


In [2]:

# fine-tuning
setup_to_finetune(bestmodel, iv3_layers_to_freeze)
 
# train the model
#checkpointer = ModelCheckpoint(filepath='saved_models/bestofbest_InceptionV3_weights', verbose=1, save_best_only=True)

bestmodel.fit_generator(datagen_train.flow(train_tensors, train_targets, batch_size=batch_size),
                    steps_per_epoch= len(train_files) // batch_size,
                    epochs=epochs, verbose=1,
                    #callbacks=[checkpointer],
                    validation_data=datagen_valid.flow(valid_tensors, valid_targets, batch_size=batch_size),
                    validation_steps=len(valid_files) // batch_size)
bestmodel.save("bestofbest1_InceptionV3.model")

### TODO: Calculate classification accuracy on the test dataset.
best_predictions = []
for img in test_files:
    best_predictions.append(bestdog_predict_breed(bestmodel,img, (299,299)))
   
# report test accuracy...test files matching test_targets

test_accuracy = 100*np.sum(np.array(best_predictions) ==
                           np.argmax(test_targets, axis=1))/len(best_predictions)
print('\nTest accuracy: %.4f%%' % test_accuracy)


Epoch 1/1
208/208 [==============================] - 3474s 17s/step - loss: 0.4618 - val_loss: 0.4696

Test accuracy: 85.1675%


In [3]:
print(train_tensors.shape)

(6680, 299, 299, 3)


In [5]:
print(train_tensors[0,])

[[[0.60784316 0.61960787 0.69411767]
  [0.6313726  0.61960787 0.69411767]
  [0.6509804  0.6431373  0.7254902 ]
  ...
  [0.57254905 0.58431375 0.65882355]
  [0.5764706  0.5882353  0.6627451 ]
  [0.5529412  0.5803922  0.6509804 ]]

 [[0.6627451  0.67058825 0.7294118 ]
  [0.6392157  0.63529414 0.69803923]
  [0.6392157  0.6509804  0.7176471 ]
  ...
  [0.5568628  0.58431375 0.654902  ]
  [0.5529412  0.57254905 0.64705884]
  [0.5411765  0.5686275  0.6392157 ]]

 [[0.5882353  0.59607846 0.654902  ]
  [0.6392157  0.63529414 0.6901961 ]
  [0.6313726  0.6431373  0.7019608 ]
  ...
  [0.5529412  0.5882353  0.654902  ]
  [0.5568628  0.5764706  0.6509804 ]
  [0.54509807 0.57254905 0.64705884]]

 ...

 [[0.13333334 0.12156863 0.05490196]
  [0.14509805 0.14117648 0.07058824]
  [0.14117648 0.1254902  0.08235294]
  ...
  [0.6117647  0.60784316 0.6627451 ]
  [0.60784316 0.6039216  0.6666667 ]
  [0.6117647  0.63529414 0.6901961 ]]

 [[0.14117648 0.12941177 0.05490196]
  [0.15686275 0.13725491 0.0627451 ]
